# Soccer Formation Analysis and Visualisation

## Prerequisite Libraries

In [1]:
import numpy as np
import pandas as pd

from src.preprocessing import preprocessing
from src.formation import formation, load_formations
from functools import partial

## Configurations

In [2]:
data_file_path = "data/raw/positions.log"
xml_file_path = "data/raw/Annotations_AtomicEvents_firstHalf.xml"

col_name =['frame_id', 'player_id','x_pos','y_pos']
length_of_field = 110.0
width_of_field = 72.0
dtypes = {'frame_id':np.int32,'player_id':np.int16,'x_pos':np.float64,'y_pos':np.float64}
form_data_file = "data/processed/form_data.csv"
frame_segment_file = "data/processed/segment.csv"

## Preprocessing

In [3]:
frame_segment_df, form_data_df = preprocessing(data_file_path, col_name, dtypes, 'SoccER', '', xml_file_path = xml_file_path)

In [4]:
form_df, sims_df= formation(form_data_file, frame_segment_file)

## Visualization

In [5]:
from bokeh.io import curdoc
from bokeh.layouts import row,column
from bokeh.models import ColumnDataSource, LabelSet, CustomJS, Title, DataTable, TableColumn, NumberFormatter
from bokeh.models.widgets import Slider, Button 
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

output_notebook()

Loading BokehJS ...

In [6]:
form_df_merged = frame_segment_df.merge(form_df)
sims_df_merged = frame_segment_df.merge(sims_df)

coor_df_a = form_data_df[form_data_df['player_id'] < 128]
form_df_a = form_df_merged[form_df_merged['player_id'] < 128]
sims_df_a = sims_df_merged[sims_df_merged['team_id'] == 0]

coor_df_b = form_data_df[form_data_df['player_id'] > 128]
form_df_b = form_df_merged[form_df_merged['player_id'] > 128]
sims_df_b = sims_df_merged[sims_df_merged['team_id'] == 1]

In [14]:
def f(doc, coor_df, form_df, sims_df, anim_speed=50, slider_steps=10):
    source_coor_df = coor_df[coor_df['frame_id'] == 1]
    source_form_df = form_df[form_df['frame_id'] == 1]
    source_sims_df = sims_df[sims_df['frame_id'] == 1].sort_values(by=['sim_score'], ascending=False)
    
    source_coor = ColumnDataSource(data=dict(x=source_coor_df['x_pos'], y=source_coor_df['y_pos'], player_id = source_coor_df['player_id']))
    source_form = ColumnDataSource(data=dict(x=source_form_df['x_pos'], y=source_form_df['y_pos'], player_id = source_form_df['player_id'] ))
    source_sims = ColumnDataSource(data=dict(score=source_sims_df['sim_score'], formation=source_sims_df['formation']))
    
    
    # https://github.com/samirak93/Game-Animation/blob/master/Animation/game_animation.py#L214
    """
    Remove plot background and alter other styles

    """
    def plot_clean(plot):

        plot.xgrid.grid_line_color = None
        plot.ygrid.grid_line_color = None
        plot.axis.major_label_text_font_size = "10pt"
        plot.axis.major_label_standoff = 0
        plot.border_fill_color = "white"
        plot.title.text_font = "times"
        plot.title.text_font_size = '10pt'
        plot.background_fill_color = "white"
        plot.title.align = 'center'
        return plot
    
    """
        Plotting
    """
    # Plotting Players' Coordinates
    plot_coor = figure(name='base',plot_height=300, plot_width=350,
              title="Players Coordinate", tools="reset,save,wheel_zoom,pan",
              x_range=(0,110), y_range=(0,72), toolbar_location="below")
    
    plot_coor.image_url(url=['image/background.png'], x=0, y=0, w=110, h=72, anchor='bottom_left')
    plot_coor.scatter('x','y',source=source_coor, size=20)
    
    labels_coor = LabelSet(x='x', y='y', text='player_id',
                  source=source_coor, y_offset=-8,
                  render_mode='canvas', text_color='black',
                  text_font_size="8pt", text_align='center')
    

    plot_coor.add_layout(labels_coor)
    plot_coor.axis.visible = False
    plot_coor = plot_clean(plot_coor)
    
    # Plotting VFS
    plot_form = figure(name='base',plot_height=300, plot_width=350,
              title="Visual Formation Summary", tools="reset,save,wheel_zoom,pan",
              x_range=(-0.2,1.2), y_range=(-0.2,0.9), toolbar_location="below")
    
    plot_form.image_url(url=['image/background.png'], x=-0.2, y=-0.2, w=1.4, h=1.1, anchor='bottom_left')
    plot_form.scatter('x','y',source=source_form, size=20)
    
    labels_form = LabelSet(x='x', y='y', text='player_id',
                  source=source_form, y_offset=-8,
                  render_mode='canvas', text_color='black',
                  text_font_size="8pt", text_align='center')
    

    plot_form.add_layout(labels_form)
    plot_form.axis.visible = False
    plot_form = plot_clean(plot_form)
    
    # Formation Similliarities Table
    columns = [
        TableColumn(field="formation", title="Formation"),
        TableColumn(field="score", title="Score", formatter=NumberFormatter(format="0.000000"))
    ]
    
    data_table = DataTable(source=source_sims, columns=columns, width = 200, height=300)
    
    """
        Slider
    """
    
    slider_start = coor_df.frame_id.min()
    slider_end = coor_df.frame_id.max()
    game_time = Slider(title="Frame ID", value=slider_start,start=slider_start, end=slider_end, step=1, sizing_mode="scale_both")

    
    def update_data(attrname, old, new):
        slider_value = np.int32(game_time.value)

        source_coor_df = coor_df[coor_df['frame_id'] == slider_value]
        source_form_df = form_df[form_df['frame_id'] == slider_value]
        source_sims_df = sims_df[sims_df['frame_id'] == slider_value].sort_values(by=['sim_score'], ascending=False)

        source_coor.data = dict(x=source_coor_df['x_pos'], y=source_coor_df['y_pos'], player_id = source_coor_df['player_id'])
        source_form.data = dict(x=source_form_df['x_pos'], y=source_form_df['y_pos'], player_id = source_form_df['player_id'])
        source_sims.data = dict(score=source_sims_df['sim_score'], formation=source_sims_df['formation'])
    
    for w in [game_time]:
        w.on_change('value', update_data)
    
    
    """
       Animation
    """
    def animate_update():

        time = game_time.value + slider_steps
        if time > coor_df.frame_id.max():
            time = coor_df.frame_id.min()
        game_time.value = time

    callback_id = None

    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = curdoc().add_periodic_callback(animate_update, anim_speed)
        else:
            button.label = '► Play'
            curdoc().remove_periodic_callback(callback_id)

    button = Button(label='► Play', width=80)
    button.on_click(animate)
    
    layout = column(row(plot_coor, plot_form, data_table), row(button, game_time))
    
    doc.add_root(layout)
    

In [15]:
plot_a = partial(f, coor_df = coor_df_a, form_df=form_df_a, sims_df= sims_df_a)

show(plot_a, notebook_url='http://localhost:8889') # https://github.com/bokeh/bokeh/blob/master/examples/howto/server_embed/notebook_embed.ipynb

In [16]:
plot_b = partial(f, coor_df = coor_df_b, form_df=form_df_b, sims_df= sims_df_b)

show(plot_b, notebook_url='http://localhost:8889') # https://github.com/bokeh/bokeh/blob/master/examples/howto/server_embed/notebook_embed.ipynb